# Hello AI World

This is `detectnet` example from ['Hello AI World'](https://github.com/dusty-nv/jetson-inference#hello-ai-world) as Jupyter Notebook.

In [ ]:
import cv2
import time
import ipywidgets
import jetson.inference
import numpy as np
import traitlets
from jetutils import GstCamera, cudaimg_to_jpeg, OUT_AS_RAW_CUDAIMG
from IPython.display import display
from sidecar import Sidecar

In [ ]:
camera = GstCamera(output_mode=OUT_AS_RAW_CUDAIMG)

In [ ]:
tic = time.time()
net = jetson.inference.detectNet('ssd-mobilenet-v2', [], 0.5)
print('net initialized in {} seconds.'.format(time.time() - tic))

Lets use ipywidgets sidecar to display both original camera input and output from detection network and some debugging information.

In [ ]:
image_original = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)
image_processed = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

debug_out = ipywidgets.Textarea(value="-"*80, disabled=True,
                               layout=ipywidgets.Layout(width='1280px'))
images_out = ipywidgets.HBox([image_original, image_processed])
confidence_slider = ipywidgets.IntSlider(value=50, min=0, max=100,
                                         description='confidence threshold',
                                         style={'description_width': 'initial'})
camera.read()

traitlets.dlink((camera, 'value'), (image_original, 'value'),
                transform=lambda x: cudaimg_to_jpeg(x, camera.width, camera.height))

control_box = ipywidgets.VBox([images_out, confidence_slider, debug_out])

_sidecar = Sidecar(title='output')
with _sidecar:
    display(control_box)

Then simply capture and overlay detections with the detection net.

In [ ]:
import IPython
ipython = IPython.get_ipython()
while True:
    ipython.kernel.do_one_iteration()
    net.SetThreshold(float(confidence_slider.value/100))
    tic = time.time()
    frame = camera.read()
    detections = net.Detect(frame, overlay='box,labels')
    fps = 1.0 / (time.time() - tic )
    debug_out.value = 'fps {}\nfps(detectnet) {}\n{}'.format(fps, 
                                                             net.GetNetworkFPS(),
                                                             ''.join(['{}: {} ({})\n'.format(x.Instance,
                                                                                             net.GetClassDesc(x.ClassID),
                                                                                             x.Confidence) 
                                                                      for x in detections]
                                                                    )
                                                             )
    image_processed.value = cudaimg_to_jpeg(frame, camera.width, camera.height)

After interrupting kernel clean up

In [ ]:
del camera
del net